In [5]:
import blocks
import theano
import theano.tensor as T
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from nems.utilities import rolling_window

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Load data

In [2]:
from scipy.io import loadmat
import os.path as path
from lnl_model_functions import gaussian

data_dir = '/Users/lmcintosh/Git/deepRGC/datasets/white_noise/'
metadata = np.load(path.join(data_dir, 'metadata.npz'))['metadata'].item()
stim  = np.memmap(path.join(data_dir, 'stim_norm.dat'), dtype=metadata['stim_norm_dtype'], 
                  mode='r', shape=metadata['stim_norm_shape'])
rates = np.memmap(path.join(data_dir, 'rates.dat'), dtype=metadata['rates_dtype'], 
                  mode='r', shape=metadata['rates_shape'])
times = np.linspace(0, 0.01*rates.shape[0], rates.shape[0])

rates_filt = np.zeros(rates.shape)
filt       = gaussian(x=np.linspace(-5,5,10), sigma=1, mu=0)
for cell in xrange(rates.shape[1]):
    rates_filt[:,cell] = np.convolve(rates[:,cell], filt, mode='same')
    
stim_sliced = stim[34:-34, 34:-34,:]
#stim_sliced = stim[52:64, 36:48]
X = rolling_window(stim_sliced, 40)
X = np.rollaxis(X, 2)
X = np.rollaxis(X, 3, 1)

Y = rates_filt[X.shape[1]:]

print (X.shape, Y.shape)

((299850, 40, 32, 32), (299850, 11))


# Create a 1 layer convolution-rectification-pool model

In [3]:
from blocks.bricks.conv import Convolutional, ConvolutionalLayer, ConvolutionalActivation
from blocks.initialization import IsotropicGaussian, Constant
from blocks.bricks import Rectifier

In [4]:
convlayer = ConvolutionalLayer(Rectifier().apply, filter_size=(11,11), num_filters=1, num_channels=40, batch_size=256, pooling_size=(10,10), image_size=(32,32), weights_init=IsotropicGaussian(), biases_init=Constant(0.01))
convlayer.initialize()

# Test main_loop

In [ ]:
from trainer import Trainer

train_me = Trainer()
learning_rate = 5e-6
reg = 0.0
dropout = 1.0
num_epochs = 1


new_model = trainer.train_memmap(X, y[:,cell], trainMask, valMask, model, three_layer_convnet, 
                                 learning_rate=learning_rate, reg=reg, update='rmsprop',
                                 dropout=dropout, num_epochs=num_epochs, batch_size=50, verbose=True,
                                 acc_frequency=5, save_plots=True)